In [2]:
from datetime import date, datetime, timedelta

import pandas as pd
import pytz
from database import Database
from email_sender import EmailSender
from excel_writer import ExcelWriter
from sqlalchemy import func, or_, select, text
from sqlalchemy.orm import aliased

db = Database()

/home/jovyan/work/database.py:34: SAWarning: Did not recognize type 'geometry' of column 'geom'
  Table(


In [3]:
select_bookings = (
    select(
        db.gwg_reservation.c.status,
        db.users_user.c.name.label("purchase_manager"),
        db.clients_operator.c.short_name.label("operator_name"),
        db.gwg_reservation.c.ref_id,
        db.gwg_reservation.c.res_id,
        db.gwg_reservation.c.bkg_ref,
        db.mapping_hotel.c.external_name.label("hotel_name"),
        db.gwg_reservation.c.guest_name,
        db.gwg_reservation.c.sales_date,
        db.gwg_reservation.c.in_date,
        db.gwg_reservation.c.out_date,
        db.gwg_reservation.c.room_type,
        db.gwg_reservation.c.meal,
        db.gwg_reservation.c.adult,
        db.gwg_reservation.c.child,
        db.gwg_reservation.c.days,
        db.gwg_reservation.c.create_date,
    )
    .outerjoin(
        db.mapping_hotel,
        db.gwg_reservation.c.hotel_id == db.mapping_hotel.c.external_code,
    )
    .outerjoin(
        db.mapping_operator,
        db.gwg_reservation.c.operator_id == db.mapping_operator.c.external_code,
    )
    .outerjoin(db.accommodation_hotel)
    .outerjoin(db.accommodation_purchase_manager)
    .outerjoin(db.users_user)
    .outerjoin(db.definitions_area)
    .outerjoin(db.definitions_region)
    .outerjoin(db.clients_operator)
    .where(
        # db.definitions_region.c.country_code == "AE",
        db.clients_operator.c.category == "IC",
        db.mapping_operator.c.external_code != 1107,
    )
    .order_by(
        db.mapping_hotel.c.external_name,
        db.gwg_reservation.c.in_date,
        db.gwg_reservation.c.ref_id,
    )
)

In [4]:
print(select_bookings)

SELECT gwg_reservation.status, users_user.name AS purchase_manager, clients_operator.short_name AS operator_name, gwg_reservation.ref_id, gwg_reservation.res_id, gwg_reservation.bkg_ref, mapping_hotel.external_name AS hotel_name, gwg_reservation.guest_name, gwg_reservation.sales_date, gwg_reservation.in_date, gwg_reservation.out_date, gwg_reservation.room_type, gwg_reservation.meal, gwg_reservation.adult, gwg_reservation.child, gwg_reservation.days, gwg_reservation.create_date 
FROM gwg_reservation LEFT OUTER JOIN mapping_hotel ON gwg_reservation.hotel_id = mapping_hotel.external_code LEFT OUTER JOIN mapping_operator ON gwg_reservation.operator_id = mapping_operator.external_code LEFT OUTER JOIN accommodation_hotel ON accommodation_hotel.id = mapping_hotel.hotel_id LEFT OUTER JOIN accommodation_purchase_manager ON accommodation_purchase_manager.id = accommodation_hotel.purchase_manager_id LEFT OUTER JOIN users_user ON users_user.id = accommodation_purchase_manager.user_id LEFT OUTER JO

In [3]:
local_tz = pytz.timezone("Asia/Dubai")
current_date = datetime.now(local_tz)

In [4]:
export = []

for destination in ["AE", "OM", "SA"]:
    outfile = (
        f'GWG_{destination}_RESLIST_{current_date.strftime("%Y_%m_%d_%H%M%S")}.xlsx'
    )

    new = pd.read_sql(
        sql=select_bookings.where(
            func.date(db.gwg_reservation.c.create_date) == current_date.date(),
            db.gwg_reservation.c.status != "Can",
            db.definitions_region.c.country_code == destination,
        ),
        con=db.engine.connect(),
    )

    ame = pd.read_sql(
        sql=select_bookings.where(
            func.date(db.gwg_reservation.c.last_modified_date) == current_date.date(),
            func.date(db.gwg_reservation.c.create_date) != current_date.date(),
            db.gwg_reservation.c.status != "Can",
            db.definitions_region.c.country_code == destination,
        ),
        con=db.engine.connect(),
    )

    can = pd.read_sql(
        sql=select_bookings.where(
            func.date(db.gwg_reservation.c.last_modified_date) == current_date.date(),
            db.gwg_reservation.c.status == "Can",
            db.definitions_region.c.country_code == destination,
        ),
        con=db.engine.connect(),
    )

    new.name = "new"
    ame.name = "ame"
    can.name = "can"

    excel_writer = ExcelWriter([new, ame, can], outfile)
    excel_writer.write_to_excel(date_column=[8, 9, 10, 16])
    excel_writer.writer.close()

    export.append(outfile)

/opt/conda/lib/python3.11/site-packages/xlsxwriter/worksheet.py:3339: UserWarning: Must have at least one data row in in add_table()
  warn("Must have at least one data row in in add_table()")
/opt/conda/lib/python3.11/site-packages/xlsxwriter/worksheet.py:3339: UserWarning: Must have at least one data row in in add_table()
  warn("Must have at least one data row in in add_table()")


In [5]:
sender = EmailSender(
    subject=f"Yesterday's Booking Intake - {(current_date - timedelta(1)).strftime('%d %b %Y')}",
    to=[
        "analyst-contracting@meetingpointuae.com",
    ],
)

sender.send_email(
    export,
    """
        <p>This is an automated report.</p>
        <p>Please do not reply.</p>
    """,
)